<a href="https://colab.research.google.com/github/BrenoMilagres/modelo_preditivo_brasileirao/blob/main/web_scraping_brasileirao2012_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Carregando módulo que possibilita utilizar um comando "mágico" para programar em R no google colab

In [ ]:
%reload_ext rpy2.ipython

#### Importando bibliotecas 

In [ ]:
%%R
library(stringr)
library(rvest)
library(glue)

R[write to console]: Loading required package: xml2



#### Acessando url do site da cbf 
#### Através da inspeção do Html do site, é possível notar que a classe aside-rodadas engloba todos os dados da tabela de jogos, então é necessário fazer a leitura desses dados utilizando os pacotes rvest e xml2.

#### Extraindo informações que são pertinentes da tabela e estruturando-as para análises posteriores. Os campos que estamos buscando são:

>Time Casa: As classes que englobam as siglas dos times da casa são pull-left e time-sigla;

>Time Fora: para o time de fora, as classes são pull-right e time-sigla;

>Placar: é agrupado pela classe partida-horario.

In [ ]:
%%R
for(i in 2012:2020) {
  
url <- glue("https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/{i}")


resultados <- url %>% 
  read_html() %>% 
  html_nodes(".aside-rodadas")

casa <- resultados %>% 
  html_nodes(".pull-left .time-sigla") %>% 
  html_text()


fora <- resultados %>% 
  html_nodes(".pull-right .time-sigla") %>% 
  html_text()


placar <-  resultados %>% 
            html_nodes(".partida-horario") %>%
             html_text() %>%
               str_extract("[0-9]{1}\ x\ [0-9]{1}")
    
rodada <- 0:(length(placar)-1) %/% 10 + 1

df <- if( i == 2012)
          { 
       
              data.frame(cbind(rodada = rodada,
                  casa = casa,
                  placar = placar,
                  fora = fora,
                  ano = rep(i,length(rodada) ) ) ) }
          else{

              data.frame(rbind(df, cbind(rodada = rodada,
                  casa = casa,
                  placar = placar,
                  fora = fora,
                  ano = rep(i,length(rodada) ) ) ))
}
 

}



#### Escrevendo um arquivo csv atrvés do dataframe gerado.

In [ ]:
%%R
write.csv(df, "Brasileirao_2012_2020.csv", col.names = TRUE, row.names = FALSE, sep = ",")

#### Tratamentos no dataframe 

In [ ]:
import pandas  as pd
df_2020 = pd.read_csv('Brasileirao_2012_2020.csv')
df_2020.tail()

,rodada,casa,placar,fora,ano
3415,38,INT,0 x 0,COR,2020
3416,38,BAH,2 x 0,SAN,2020
3417,38,ATH,2 x 0,SPO,2020
3418,38,CEA,2 x 1,BOT,2020
3419,38,ATL,3 x 1,COR,2020


Separando placar 

In [ ]:
gols = df_2020['placar'].str.split('x',n=1,expand=True)
df_2020['MandanteGols'] = gols[0]
df_2020['VisitanteGols'] = gols[1]
df_2020.head()

,rodada,casa,placar,fora,ano,MandanteGols,VisitanteGols
0,1,FOR,0 x 2,ATH,2020,0,2
1,1,COR,0 x 1,INT,2020,0,1
2,1,SPO,3 x 2,CEA,2020,3,2
3,1,FLA,0 x 1,ATL,2020,0,1
4,1,SAN,1 x 1,RED,2020,1,1
